![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Module 6: Capstone Comparison

**⏱️ Time:** 30 minutes

## 🎯 Learning Objectives

By the end of this module, you will:

1. **Compare** Stage 4 (ReAct) vs Stage 6 (Full Memory) agents
2. **Understand** the trade-offs between different agent architectures
3. **Know** when to use which approach in production
4. **Have** a clear path for continued learning

---

## 📚 Part 1: The Progressive Agents Journey (10 min)

### What We've Built

Throughout this workshop, we've progressively added capabilities:

| Stage | Name | Key Feature | Token Usage |
|-------|------|-------------|-------------|
| 1 | Baseline RAG | Raw JSON context | ~6,133 tokens |
| 2 | Context Engineered | Clean, transformed context | ~539 tokens |
| 3 | Full Agent | LangGraph + tool calling | ~600 tokens |
| 4 | Hybrid Search + ReAct | Visible reasoning | ~700 tokens |
| 5 | Working Memory | Session continuity | ~800 tokens |
| 6 | Full Memory | Cross-session persistence | ~900 tokens |

### The Key Insight

**Context engineering isn't just about reducing tokens—it's about using them wisely.**

Stage 6 uses more tokens than Stage 2, but provides:
- Conversation continuity
- Personalization
- Transparent reasoning
- Cross-session memory

In [1]:
# Setup
import os
import sys
from pathlib import Path

repo_root = Path.cwd().parent
src_path = repo_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Load environment variables
from dotenv import load_dotenv
load_dotenv()  # Try current dir first
load_dotenv(repo_root / ".env")  # Then try parent

print("✅ Setup complete!")

✅ Setup complete!


---

## 📚 Part 2: Stage 4 vs Stage 6 Comparison (15 min)

### Stage 4: ReAct with Hybrid Search

**Features:**
- Visible reasoning (Thought → Action → Observation)
- Hybrid search (exact + semantic)
- Single tool: `search_courses`
- No memory (stateless)

**Best for:**
- Simple Q&A applications
- When debugging is important
- When you don't need personalization

### Stage 6: Full Memory

**Features:**
- Everything from Stage 4, plus:
- Working memory (session continuity)
- Long-term memory tools (`search_memories`, `store_memory`)
- 3 tools for LLM to choose from
- Cross-session personalization

**Best for:**
- Personalized assistants
- Multi-turn conversations
- When user context matters

In [2]:
# Comparison table
comparison = {
    "Feature": ["Reasoning", "Search", "Tools", "Working Memory", "Long-term Memory", "Personalization", "Complexity"],
    "Stage 4": ["Visible (ReAct)", "Hybrid", "1", "❌", "❌", "❌", "Medium"],
    "Stage 6": ["Visible (ReAct)", "Hybrid", "3", "✅", "✅", "✅", "High"]
}

print("Stage 4 vs Stage 6 Comparison")
print("="*60)
print(f"{'Feature':<20} {'Stage 4':<20} {'Stage 6':<20}")
print("-"*60)
for i in range(len(comparison["Feature"])):
    print(f"{comparison['Feature'][i]:<20} {comparison['Stage 4'][i]:<20} {comparison['Stage 6'][i]:<20}")

Stage 4 vs Stage 6 Comparison
Feature              Stage 4              Stage 6             
------------------------------------------------------------
Reasoning            Visible (ReAct)      Visible (ReAct)     
Search               Hybrid               Hybrid              
Tools                1                    3                   
Working Memory       ❌                    ✅                   
Long-term Memory     ❌                    ✅                   
Personalization      ❌                    ✅                   
Complexity           Medium               High                


### Live Demo: Running the Agents

Let's see both agents in action with the same query.

In [3]:
# Demo: Stage 4 agent (ReAct without memory)
# Run from terminal: cd progressive_agents/stage4_hybrid_search && python cli.py --show-reasoning "What ML courses do you recommend?"

print("Stage 4 Example Output:")
print("="*60)
print("""
🧠 Reasoning Trace:
================================================================================
💭 Thought: The user wants ML course recommendations. I'll search for ML courses.

🔧 Action: search_courses
   Input: {"query": "machine learning", "intent": "GENERAL", "search_strategy": "hybrid"}
👁️  Observation: Found 5 courses: CS002 (ML Fundamentals), CS006 (Deep Learning)...

💭 Thought: I have the course information. I can provide recommendations.

✅ FINISH
================================================================================

Answer: Here are some machine learning courses I recommend:
1. CS002 - Machine Learning Fundamentals (Beginner)
2. CS006 - Deep Learning and Neural Networks (Advanced)
...""")

Stage 4 Example Output:

🧠 Reasoning Trace:
💭 Thought: The user wants ML course recommendations. I'll search for ML courses.

🔧 Action: search_courses
   Input: {"query": "machine learning", "intent": "GENERAL", "search_strategy": "hybrid"}
👁️  Observation: Found 5 courses: CS002 (ML Fundamentals), CS006 (Deep Learning)...

💭 Thought: I have the course information. I can provide recommendations.

✅ FINISH

Answer: Here are some machine learning courses I recommend:
1. CS002 - Machine Learning Fundamentals (Beginner)
2. CS006 - Deep Learning and Neural Networks (Advanced)
...


In [4]:
# Demo: Stage 6 agent (Full memory)
# Run from terminal: cd progressive_agents/stage6_full_memory && python cli.py --student-id alice --show-reasoning "I prefer online courses. What ML courses do you recommend?"

print("Stage 6 Example Output:")
print("="*60)
print("""
🧠 Reasoning Trace:
================================================================================
💭 Thought: The user expressed a preference AND asked for recommendations.
            I should store the preference first, then search for courses.

🔧 Action: store_memory
   Input: {"text": "Student prefers online courses", "topics": ["preferences", "format"]}
👁️  Observation: Successfully stored user preference.

💭 Thought: Now I should search for ML courses.

🔧 Action: search_courses
   Input: {"query": "machine learning", "intent": "GENERAL", "search_strategy": "hybrid"}
👁️  Observation: Found 5 courses: CS002 (ML Fundamentals, Online), ...

💭 Thought: I have courses and stored the preference. I can provide personalized recommendations.

✅ FINISH
================================================================================

Answer: I've noted your preference for online courses! Here are ML courses available online:
1. CS002 - Machine Learning Fundamentals (Online, Beginner)
2. CS006 - Deep Learning and Neural Networks (Online, Advanced)
...""")

Stage 6 Example Output:

🧠 Reasoning Trace:
💭 Thought: The user expressed a preference AND asked for recommendations.
            I should store the preference first, then search for courses.

🔧 Action: store_memory
   Input: {"text": "Student prefers online courses", "topics": ["preferences", "format"]}
👁️  Observation: Successfully stored user preference.

💭 Thought: Now I should search for ML courses.

🔧 Action: search_courses
   Input: {"query": "machine learning", "intent": "GENERAL", "search_strategy": "hybrid"}
👁️  Observation: Found 5 courses: CS002 (ML Fundamentals, Online), ...

💭 Thought: I have courses and stored the preference. I can provide personalized recommendations.

✅ FINISH

Answer: I've noted your preference for online courses! Here are ML courses available online:
1. CS002 - Machine Learning Fundamentals (Online, Beginner)
2. CS006 - Deep Learning and Neural Networks (Online, Advanced)
...


### Key Differences Observed

| Aspect | Stage 4 | Stage 6 |
|--------|---------|--------|
| **Tool Calls** | 1 (search_courses) | 2 (store_memory + search_courses) |
| **Personalization** | None | Stores and uses preferences |
| **Cross-Session** | No memory | Remembers preferences |
| **Response Quality** | Generic | Personalized |

---

## 📚 Part 3: Production Patterns (5 min)

### When to Use Each Stage

| Use Case | Recommended Stage | Why |
|----------|-------------------|-----|
| Simple FAQ bot | Stage 2-3 | No memory needed |
| Debugging/testing | Stage 4 | Visible reasoning |
| Customer support | Stage 5 | Session continuity |
| Personal assistant | Stage 6 | Full personalization |

### Cost vs. Capability Trade-offs

```
Cost/Complexity →
                    Stage 6 (Full Memory)
                   /
                  Stage 5 (Working Memory)
                 /
                Stage 4 (ReAct)
               /
              Stage 3 (Agent)
             /
            Stage 2 (Context Engineered)
           /
          Stage 1 (Baseline)
         ↓
Capability →
```

**Key Insight:** Start simple, add complexity only when needed.

---

## 🎯 Workshop Summary

### What You've Learned

| Module | Key Concepts |
|--------|-------------|
| **1. Foundations** | Four context types, token budgeting, 91% reduction |
| **2. RAG Essentials** | Vector embeddings, semantic search, progressive disclosure |
| **3. Memory Systems** | Working memory, long-term memory, automatic compression |
| **4. Building Agents** | LangGraph, tool calling, memory tools |
| **5. Capstone** | Stage comparison, production patterns |

### The Context Engineering Mindset

1. **Context is everything** - AI quality depends on context quality
2. **Tokens are precious** - Optimize for signal, not noise
3. **Memory enables intelligence** - Stateless AI is limited AI
4. **Tools extend capabilities** - Let LLMs decide when to use them
5. **Start simple, add complexity** - Don't over-engineer

---

## 📖 Next Steps

### Continue Learning

**Full Course (15-20 hours):**
- `notebooks/` - Complete 11-notebook curriculum
- `progressive_agents/` - All 6 stages with detailed READMEs

**Advanced Topics:**
- Semantic tool selection (scaling to many tools)
- Custom compression strategies
- Production deployment patterns

### Resources

| Resource | Link |
|----------|------|
| Redis Vector Search | https://redis.io/docs/stack/search/reference/vectors/ |
| LangGraph | https://langchain-ai.github.io/langgraph/ |
| Agent Memory Server | https://github.com/redis/agent-memory-server |

### Try It Yourself

```bash
# Run Stage 4 agent
cd progressive_agents/stage4_hybrid_search
python cli.py --show-reasoning "What are the prerequisites for CS009?"

# Run Stage 6 agent
cd progressive_agents/stage6_full_memory
python cli.py --student-id your_name --show-reasoning "I prefer online courses"
```

---

## 🎉 Congratulations!

You've completed the **Context Engineering Workshop**!

You now understand:
- ✅ The four types of context
- ✅ How to optimize token usage (91% reduction!)
- ✅ RAG with progressive disclosure
- ✅ Working and long-term memory
- ✅ Building agents with LangGraph
- ✅ When to use different agent architectures

**Go build something amazing!** 🚀